In [ ]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [ ]:
data_train = pd.read_csv('data/train_preprocessed.csv')
data_train.columns

In [ ]:
data_train.drop(index=35065, inplace=True)

In [ ]:
# Assuming `data` is your DataFrame and `id` is the timestamp column
data_train['id'] = pd.to_datetime(data_train['id'])
data_train = data_train.copy()
data_train['time_idx'] = (data_train['id'] - data_train['id'].min()) / pd.Timedelta(hours=1)
data_train['time_idx'] = data_train['time_idx'].astype(int)
data_train['location']="Montsouris"
data_train['car_flow'] = data_train['car_flow'].fillna(-1)

In [ ]:
data_train

In [ ]:
data_train.columns

In [ ]:
data_train['holiday_type'] = np.where(data_train['is_holiday'] == 1, 'holiday', 
                              np.where(data_train['is_jour_ferie'] == 1, 'ferie', np.nan))
data_train = data_train.drop(columns=['is_holiday','is_jour_ferie'])
data_train['holiday_type']

In [ ]:
data_train.columns

In [ ]:
max_prediction_length = 502
max_encoder_length = 365*24*3

In [ ]:
training_cutoff = data_train["time_idx"].max() - max_prediction_length
training_cutoff

In [ ]:
data_train['Year'] = data_train['Year'].astype(str)
data_train['Month'] = data_train['Month'].astype(str)
data_train['Day'] = data_train['Day'].astype(str)
data_train['is_weekend'] = data_train['is_weekend'].astype(str)
data_train['DayOfYear'] = data_train['DayOfYear'].astype(str)
data_train['HourOfDay'] = data_train['HourOfDay'].astype(str)
data_train['Weekday'] = data_train['Weekday'].astype(str)

In [ ]:
training = TimeSeriesDataSet(
    data_train[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10','valeur_PM25'],
    group_ids=["location"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    time_varying_known_categoricals=['Year','Month', 'Day', 'is_weekend','DayOfYear', 'HourOfDay', 'Weekday','holiday_type'],
    variable_groups={"special_days": ["holiday_type"]},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["time_idx", 'DayOfYear_sin', 'DayOfYear_cos', 'HourOfDay_sin', 'HourOfDay_cos','Weekday_sin', 'Weekday_cos','precipitation', 'wind_speed', 'temperature', 'humidity','pressure', 'visibility', 'global_solar_radiation'],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        'valeur_NO2',
        'valeur_CO',
        'valeur_O3',
        'valeur_PM10',
        'valeur_PM25',
        'car_flow'
    ]
)

# Following is not working

In [ ]:
tft = TemporalFusionTransformer.from_dataset(
    dataset,  # the dataset we defined previously
    learning_rate=0.03,
    hidden_size=16,  # model capacity
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[1,1,1,1,1],  # Number of target variables
    loss=QuantileLoss()  # Use QuantileLoss for multi-output forecasting
)

In [ ]:
trainer = Trainer(
    max_epochs=30,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices="auto" if torch.cuda.is_available() else 1  # Set to 1 for CPU
)

In [ ]:
trainer.fit(tft, train_dataloaders=train_dataloader)


trainer = Trainer(max_epochs=30, gpus=1 if torch.cuda.is_available() else 0)  # Use 1 GPU if available

In [ ]:
data_train["id"] = pd.to_datetime(data_train["id"])
features = ['id', 'is_holiday', 'is_jour_ferie', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation', 'Year', 'is_weekend', 'DayOfYear',
       'HourOfDay', 'DayOfYear_sin', 'DayOfYear_cos', 'HourOfDay_sin',
       'HourOfDay_cos', 'Weekday_sin', 'Weekday_cos']
data_train["location"] = "Montsouris"
data_train['time_idx'] = ((data_train['id'] - data_train['id'].min()).dt.total_seconds() // 3600).astype("int")
data_train.shape

In [ ]:
data_train.drop_duplicates()

In [ ]:
max_encoder_length = 7 * 24
max_prediction_length = 502  # Forecast 502 hours into the future

In [ ]:
data_train.drop(index=35065, inplace=True)

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

split_idx = int(data_train["time_idx"].max() * 0.8)  # 80% for training, adjust as needed

train_data = data_train[data_train["time_idx"] <= split_idx]
val_data = data_train[data_train["time_idx"] > split_idx]

train_dataset = TimeSeriesDataSet(
    train_data,
    time_idx="time_idx",
    target=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    group_ids=["location"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    time_varying_known_reals=features,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

val_dataset = TimeSeriesDataSet(
    val_data,
    time_idx="time_idx",
    target=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    group_ids=["location"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    time_varying_known_reals=features,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


In [ ]:
class MultiLoss:
    def __init__(self, losses):
        self.losses = losses

    def __call__(self, y_pred, y_true):
        total_loss = 0
        for i, loss_fn in enumerate(self.losses):
            total_loss += loss_fn(y_pred[..., i], y_true[..., i])
        return total_loss

In [ ]:
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_lightning import Trainer
from pytorch_forecasting.metrics import QuantileLoss

# Define TFT model
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=0.03,  # You may adjust this
    hidden_size=16,  # Size of the network layers
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[1, 1, 1, 1, 1], 
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

print(f"Number of parameters in model: {tft.size()/1e3:.1f}k")


In [ ]:
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer

In [ ]:
import pytorch_lightning as pl
isinstance(tft, pl.LightningModule)

In [ ]:
trainer = Trainer(
    max_epochs=30,
    gradient_clip_val=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5)],
    precision=16 if torch.cuda.is_available() else 32,
    log_every_n_steps=10
)

In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataset.to_dataloader(train=True, batch_size=64, num_workers=4),
    val_dataloaders=val_dataset.to_dataloader(train=False, batch_size=64, num_workers=4),
)


In [ ]:
import pytorch_lightning as pl
import torch

class TFTLightningWrapper(pl.LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.model = tft_model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.model.loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.model.loss(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.03)


In [ ]:
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

# Instantiate the TemporalFusionTransformer model
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[1, 1, 1, 1, 1],  # Adjusted for each target variable
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Wrap the model in the custom LightningModule wrapper
tft_wrapped = TFTLightningWrapper(tft)


In [ ]:
trainer = pl.Trainer(
    max_epochs=30
)

trainer.fit(
    tft_wrapped,
    train_dataloaders=train_dataset.to_dataloader(train=True, batch_size=64, num_workers=4),
    val_dataloaders=val_dataset.to_dataloader(train=False, batch_size=64, num_workers=4),
)
